In [2]:
from datasets import load_dataset, load_metric, Dataset
import pandas as pd

df_train = pd.read_csv("/home/ubuntu/uzair/fine_tuning_stt/train_df.csv")
df_test = pd.read_csv("/home/ubuntu/uzair/fine_tuning_stt/test_df.csv")


train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

/home/ubuntu/anaconda3/envs/med_stt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_dataset

Dataset({
    features: ['filename', 'transcription'],
    num_rows: 5895
})

In [4]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(train_dataset.remove_columns(["filename"]), num_examples=20)

,transcription
0,I tried alot of cream but i still feel the same skin problem
1,I have severe shoulder pain
2,I feel hurts in my heart
3,The acne was causing me to feel embarrassed.
4,It hurts when I raise my arm up
5,My stomach feels starts hurting but then it feels better after I eat something mild.
6,"I think there's something wrong with my wound, it does not seem to heal like it should."
7,Oh my heart hurts me I tried to be calm and I can't
8,When I tried to be warm and wear more clothes I found that I was still cold
9,What's the symptoms of appendicitis?


In [6]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch


In [7]:
show_random_elements(train_dataset.remove_columns(["filename"]))

,transcription
0,I have a pain internal
1,my temperature dropped and my body get synosed
2,I feel a pain in my back
3,It itches inside my ears.
4,My foot had been aching since last Tuesday.
5,I can`t stop coughing.
6,Chronic bone disease at the level of the doubles of the lower Estremidades
7,I feel pain when i think of her
8,When i'm awake in the morning I feel strange and have vertigo
9,I have a sharp pain in my lower stomach.


In [8]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [9]:
vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)

Map: 100%|██████████| 766/766 [00:00<00:00, 256145.81 examples/s]


In [10]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [11]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}


In [12]:
vocab_dict

{'c': 0,
 '1': 1,
 'l': 2,
 'k': 3,
 '√': 4,
 ' ': 5,
 'x': 6,
 'd': 7,
 '0': 8,
 'D': 9,
 'u': 10,
 'Y': 11,
 't': 12,
 'o': 13,
 'C': 14,
 'y': 15,
 'g': 16,
 'v': 17,
 'E': 18,
 '2': 19,
 "'": 20,
 'M': 21,
 '!': 22,
 'H': 23,
 'N': 24,
 'P': 25,
 'L': 26,
 '?': 27,
 '5': 28,
 ',': 29,
 '3': 30,
 'm': 31,
 '.': 32,
 'I': 33,
 'q': 34,
 'p': 35,
 'F': 36,
 'j': 37,
 'i': 38,
 '-': 39,
 'J': 40,
 'a': 41,
 ':': 42,
 'e': 43,
 '7': 44,
 'b': 45,
 'S': 46,
 'z': 47,
 'T': 48,
 '\n': 49,
 'f': 50,
 'r': 51,
 'B': 52,
 'A': 53,
 'O': 54,
 's': 55,
 'w': 56,
 'W': 57,
 'h': 58,
 '`': 59,
 'G': 60,
 'n': 61,
 'ï': 62}

In [13]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [14]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

65

In [15]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [16]:
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token=".")

In [17]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [18]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [19]:
processor.save_pretrained(".")

[]

In [20]:
import torchaudio
import os
from torchaudio.transforms import Resample

train_dir   = "/home/ubuntu/uzair/fine_tuning_stt/dataset/recordings/train"
test_dir = "/home/ubuntu/uzair/fine_tuning_stt/dataset/recordings/test"
def speech_file_to_array_fn_train(batch):
    # Load the audio file
    file_path = train_dir + os.sep + batch["filename"]
    speech_array, sampling_rate = torchaudio.load(file_path)

    # Resample the audio to 16000
    resample = Resample(orig_freq=sampling_rate, new_freq=16000)
    resampled_speech_array = resample(speech_array)

    # Update the batch with resampled speech array and other relevant information
    batch["speech"] = resampled_speech_array[0].numpy()
    batch["sampling_rate"] = 16000
    batch["target_text"] = batch["transcription"]
    return batch

In [21]:
def speech_file_to_array_fn_test(batch):
    # Load the audio file
    file_path = test_dir + os.sep + batch["filename"]
    speech_array, sampling_rate = torchaudio.load(file_path)

    # Resample the audio to 16000
    resample = Resample(orig_freq=sampling_rate, new_freq=16000)
    resampled_speech_array = resample(speech_array)

    # Update the batch with resampled speech array and other relevant information
    batch["speech"] = resampled_speech_array[0].numpy()
    batch["sampling_rate"] = 16000
    batch["target_text"] = batch["transcription"]
    return batch

In [22]:
train_dataset = train_dataset.map(speech_file_to_array_fn_train, remove_columns=train_dataset.column_names)


Map: 100%|██████████| 5895/5895 [00:40<00:00, 147.11 examples/s]


In [23]:
test_dataset = test_dataset.map(speech_file_to_array_fn_test, remove_columns=test_dataset.column_names)

Map: 100%|██████████| 766/766 [00:04<00:00, 170.11 examples/s]


In [24]:
rand_int = random.randint(0, len(train_dataset))
import numpy as np
print("Target text:", train_dataset[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_dataset[rand_int]["speech"]).shape)
print("Sampling rate:", train_dataset[rand_int]["sampling_rate"])

Target text: Surgical wound infections
Input array shape: (42326,)
Sampling rate: 16000


In [25]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [26]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names, batch_size=16, num_proc=16, batched=True)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names, batch_size=16, num_proc=16, batched=True)

Map (num_proc=16):   0%|          | 0/5895 [00:00<?, ? examples/s]/home/ubuntu/anaconda3/envs/med_stt/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Map (num_proc=16):   1%|          | 64/5895 [00:04<05:58, 16.28 examples/s]/home/ubuntu/anaconda3/envs/med_stt/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Map (num_proc=16):   6%|▌         | 336/5895 [00:08<01:18, 70.69 examples/s]/ho

In [27]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [28]:
wer_metric = load_metric("wer")

/tmp/ipykernel_34919/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/home/ubuntu/anaconda3/envs/med_stt/lib/python3.8/site-packages/datasets/load.py:753: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [29]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [30]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

/home/ubuntu/anaconda3/envs/med_stt/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
model.freeze_feature_extractor()

/home/ubuntu/anaconda3/envs/med_stt/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1919: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [32]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/home/ubuntu/uzair/fine_tuning_stt/output_model_wav2vec2",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=400, #this would mean every 400 steps model gets saved which also means Google drive gets full
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  # learning_rate=0.1, # this is just for demo
  warmup_steps=500,
  save_total_limit=2,
)

In [33]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [34]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

In [35]:
trainer.train()

/home/ubuntu/anaconda3/envs/med_stt/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
400,6.040600,inf,0.999253
800,1.399400,inf,0.368277
1200,0.398500,inf,0.275523
1600,0.277600,inf,0.253362


/home/ubuntu/anaconda3/envs/med_stt/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ubuntu/anaconda3/envs/med_stt/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ubuntu/anaconda3/envs/med_stt/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can

TrainOutput(global_step=1840, training_loss=1.8000462366187053, metrics={'train_runtime': 3617.3741, 'train_samples_per_second': 16.296, 'train_steps_per_second': 0.509, 'total_flos': 8.258655455683359e+18, 'train_loss': 1.8000462366187053, 'epoch': 9.97})

In [38]:
trainer.save_model("wav2vec2_asr")
